# Setup

In [4]:
from parameters.parameters import prune_quantiles, prune_measure

debug = True # If True, will produce plots and/or verbose output to double-check

import sys
sys.path.append("..")

# Path
print("Current working directory: ", sys.path[0])
%run -i "../parameters/parameters.py"
%run -i "../src/path.py"
%run -i "../src/setup.py"
%run -i "../src/functions.py"
%run -i "greedy_triangulation.py"
%run -i "graph_similarity.py"

%load_ext watermark
%watermark -n -v -m -g -iv

Current working directory:  /Users/jaborup/Projects/continuous-bike-growth
Loaded parameters.

Loaded PATH.



=== Cities ===
{   'montreal': {   'countryid': 'can',
                    'name': 'Montreal',
                    'nominatimstring': 'montreal (06)'}}


Setup finished.

Loaded functions.

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
numpy      1.26.4
shapely    2.0.4
osmnx      2.0.0b3
fiona      1.9.5
osgeo.gdal 3.8.5
pandas     2.2.3
geojson    2.5.0
geopandas  1.0.1
watermark  2.0.2
igraph     0.11.6
csv        1.0
pyproj     3.6.1
matplotlib 3.8.4
networkx   3.3
Sun Nov 17 2024 

CPython 3.12.7
IPython 8.6.0

compiler   : Clang 17.0.6 
system     : Darwin
release    : 23.6.0
machine    : arm64
processor  : arm
CPU cores  : 8
interpreter: 64bit
Git hash   : 995eb3a88f7d9d57d381e74cd1aa61925f88df12


# Run the program

In [5]:
for placeid, placeinfo in tqdm(cities.items(), desc="Cities"):
    print(placeid + ": Generating graph and POIs")

    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
    with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
        nnids = [int(line.rstrip()) for line in f]

    subgraph_percentage = 0.6
    prune_quantiles = [0.5]
    for prune_measure in ["betweenness", "closeness", "random"]:
        print(f"Prune measure: {prune_measure}")
        data = [
            "sum_of_errors",
            "continuous_vertices",
            "bikengrowth_vertices",
            "continuous_disconnected_points",
            "bikengrowth_disconnected_points",
        ]

        (GTs, GT_abstracts) = _greedy_triangulation_routing(G_carall, nnids, prune_quantiles, prune_measure)

        for i in range(100):
            if (i + 1) % 25 == 0:
                print(f"Running iteration {i + 1}")
            if prune_measure == "random":
                (GTs, GT_abstracts) = _greedy_triangulation_routing(G_carall, nnids, prune_quantiles, prune_measure)

            gts, abstract_gts = greedy_triangulation_in_steps(G_carall, nnids, subgraph_percentage, prune_quantiles, prune_measure)

            for prune_quantile, gt, szell_gt in zip(prune_quantiles, gts, GTs):
                error, our_disconnected_points, szell_disconnected_points = sum_of_errors_pois(gt, szell_gt, nnids)
                data.append([error, gt.vcount(), szell_gt.vcount(), our_disconnected_points, szell_disconnected_points])

        print(f"Writing data to {placeid}_{prune_measure}.csv")
        with open(f'{placeid}_{prune_measure}.csv', 'w') as f:
            writer = csv.writer(f)
            writer.writerows(data)

Cities:   0%|          | 0/1 [00:00<?, ?it/s]

montreal: Generating graph and POIs


/Users/jaborup/Projects/continuous-bike-growth/src/functions.py:345: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  n = pd.read_csv(p + prefix + '_nodes.csv')
/Users/jaborup/Projects/continuous-bike-growth/src/functions.py:346: DtypeWarning: Columns (5,9,16) have mixed types. Specify dtype option on import or set low_memory=False.
  e = pd.read_csv(p + prefix + '_edges.csv')


Prune measure: betweenness


Greedy triangulation:   0%|          | 0/1 [00:00<?, ?it/s]

...

Greedy triangulation on 60.0% subgraph:   0%|          | 0/1 [00:00<?, ?it/s]

Writing data to montreal_betweenness.csv
Prune measure: closeness


Greedy triangulation:   0%|          | 0/1 [00:00<?, ?it/s]

Greedy triangulation on 60.0% subgraph:   0%|          | 0/1 [00:00<?, ?it/s]

ValueError: no such vertex

In [ ]:
for placeid, placeinfo in tqdm(cities.items(), desc="Cities"):
    result_file = open(PATH["results"] + placeid + "/" + filename + ".pickle", 'rb')
    res = pickle.load(result_file)
    szell_gts = res["GTs"]